In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import math as m
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
def Angle(x1, y1, x2, y2, x3, y3):
    
    if x1 == x2 or x2 == x3 :
        degree = 90
    elif y1 == y2 or y2 == y3:
        degree = 0
    else :
        theta = m.atan( (y1- y2) / ( x1- x2)) - m.atan( (y3 - y2) / (x3 - x2))
        degree = int(180/m.pi)*theta

    
    if degree <0 :
        degree = degree*(-1)
    return degree

In [15]:

def lat_TR():
    def calculate_angle(a,b,c):
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End
        
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        
        if angle >180.0:
            angle = 360-angle
            
        return angle


    def Angle(x1, y1, x2, y2, x3, y3):
        
        if x1 == x2 or x2 == x3 :
            degree = 90
        elif y1 == y2 or y2 == y3:
            degree = 0
        else :
            theta = m.atan( (y1- y2) / ( x1- x2)) - m.atan( (y3 - y2) / (x3 - x2))
            degree = int(180/m.pi)*theta

        
        if degree <0 :
            degree = degree*(-1)
        return degree 
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
                

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <30 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                    
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                
                        
                elif len(b) < len(a) :


                    if angle > 70:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                    
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
            
                        
                elif len(c) < len(b) :


                    if angle > 160 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
    
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle <70:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
            
                        
                elif len(e) < len(d) :
        

                    if angle <40 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
        
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)

                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



        df = pd.DataFrame([f1,f2, f3, f4, f5])
        
        df.columns = ["leftElbowAngle0","leftShoulderAngle0","rightElbowAngle0","rightShoulderAngle0",
                "leftElbowAngle5","leftShoulderAngle5","rightElbowAngle5","rightShoulderAngle5",
                "leftElbowAngle8","leftShoulderAngle8","rightElbowAngle8","rightShoulderAngle8",
                "leftElbowAngle12","leftShoulderAngle12","rightElbowAngle12","rightShoulderAngle12",
                "leftElbowAngle15","leftShoulderAngle15","rightElbowAngle15","rightShoulderAngle15"]
        scalar = StandardScaler()
        x_data_scaled = scalar.fit_transform(df)
        model = load_model('latpullpredict1.h5')

        a = np.array(x_data_scaled)
        b = model.predict(a)
        c = np.argmax(b, axis=1)
        
    val=[]
    acc=0

    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("상체가 불안정하며, 손목이 꺾였습니다")
        elif c[i]==2:
            val.append("상체가 불안정하며, 축 평행이 불량합니다.")
        elif c[i]==3:
            val.append("상체가 불안정하며, 기구 당김이 부족합니다")
   
    print(val)
    
    insertLat_TR(val, nickname, acc)

In [18]:
lat_TR()


1/1 [==============================] - 0s 77ms/step
[0 1 0 3 3]
['정자세입니다.', '상체가 불안정하며, 손목이 꺾였습니다', '정자세입니다.', '상체가 불안정하며, 기구 당김이 부족합니다', '상체가 불안정하며, 기구 당김이 부족합니다']


In [232]:
df = pd.DataFrame([f1,f2, f3, f4, f5])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,26.535607,38.433041,23.679304,144.704434,80.505894,103.243445,86.486386,72.837065,169.307868,174.662404,2.913315,174.446006,57.254971,71.415704,49.362116,106.249414,39.076279,48.307030,34.911115,133.339331
1,27.327084,29.774179,152.475059,149.743850,73.036542,90.488890,76.028157,84.040839,159.358377,159.831728,160.740311,7.905232,62.812274,99.283991,60.231978,97.876572,37.347741,124.674725,42.127676,114.625308
2,21.868941,146.717387,22.455172,145.605412,73.435516,90.310218,69.774760,86.996626,165.475297,11.584551,4.494084,0.243352,66.904380,95.583114,64.720152,96.621695,39.422266,42.087981,143.676678,138.127203
3,29.620260,43.640631,29.155499,139.125913,105.638939,46.004744,106.963868,51.075385,163.583617,171.690619,172.844557,2.090412,66.520479,79.588984,70.655576,86.090590,36.338175,133.688697,35.417684,127.958699
4,29.815118,140.622940,29.461600,136.951900,90.449749,70.601377,80.028403,78.655374,162.449018,169.730493,168.806635,0.087361,61.260659,78.642772,57.089452,95.945733,38.453954,50.624359,34.495126,131.425678


In [233]:
df.columns = ["leftElbowAngle0","leftShoulderAngle0","rightElbowAngle0","rightShoulderAngle0",
             "leftElbowAngle5","leftShoulderAngle5","rightElbowAngle5","rightShoulderAngle5",
             "leftElbowAngle8","leftShoulderAngle8","rightElbowAngle8","rightShoulderAngle8",
             "leftElbowAngle12","leftShoulderAngle12","rightElbowAngle12","rightShoulderAngle12",
             "leftElbowAngle15","leftShoulderAngle15","rightElbowAngle15","rightShoulderAngle15"]

In [234]:
df

,leftElbowAngle0,leftShoulderAngle0,rightElbowAngle0,rightShoulderAngle0,leftElbowAngle5,leftShoulderAngle5,rightElbowAngle5,rightShoulderAngle5,leftElbowAngle8,leftShoulderAngle8,rightElbowAngle8,rightShoulderAngle8,leftElbowAngle12,leftShoulderAngle12,rightElbowAngle12,rightShoulderAngle12,leftElbowAngle15,leftShoulderAngle15,rightElbowAngle15,rightShoulderAngle15
0,26.535607,38.433041,23.679304,144.704434,80.505894,103.243445,86.486386,72.837065,169.307868,174.662404,2.913315,174.446006,57.254971,71.415704,49.362116,106.249414,39.076279,48.307030,34.911115,133.339331
1,27.327084,29.774179,152.475059,149.743850,73.036542,90.488890,76.028157,84.040839,159.358377,159.831728,160.740311,7.905232,62.812274,99.283991,60.231978,97.876572,37.347741,124.674725,42.127676,114.625308
2,21.868941,146.717387,22.455172,145.605412,73.435516,90.310218,69.774760,86.996626,165.475297,11.584551,4.494084,0.243352,66.904380,95.583114,64.720152,96.621695,39.422266,42.087981,143.676678,138.127203
3,29.620260,43.640631,29.155499,139.125913,105.638939,46.004744,106.963868,51.075385,163.583617,171.690619,172.844557,2.090412,66.520479,79.588984,70.655576,86.090590,36.338175,133.688697,35.417684,127.958699
4,29.815118,140.622940,29.461600,136.951900,90.449749,70.601377,80.028403,78.655374,162.449018,169.730493,168.806635,0.087361,61.260659,78.642772,57.089452,95.945733,38.453954,50.624359,34.495126,131.425678


In [235]:
scalar = StandardScaler()
x_data_scaled = scalar.fit_transform(df)
model = load_model('latpullpredict1.h5')

a = np.array(x_data_scaled)
b = model.predict(a)
c = np.argmax(b, axis=1)
val=[]
for i in range(len(c)):
    if c[i]==0:
        val.append("정자세입니다.")
    elif c[i]==1:
        val.append("상체가 불안정하며, 손목이 꺾였습니다")
    elif c[i]==2:
        val.append("상체가 불안정하며, 축 평행이 불량합니다.")
    elif c[i]==3:
        val.append("상체가 불안정하며, 기구 당김이 부족합니다")
print(c)

[0 0 1 3 3]


In [236]:
val

['정자세입니다.',
 '정자세입니다.',
 '상체가 불안정하며, 손목이 꺾였습니다',
 '상체가 불안정하며, 기구 당김이 부족합니다',
 '상체가 불안정하며, 기구 당김이 부족합니다']